# Trust in Multi-Agent Systems
Welcome to our lesson on trust and explainability in AI! 

Before we get started, remember to run this cell to get everything set up correctly:

In [45]:
%load_ext autoreload
%autoreload 2
from utils import prep_browser, run_pyperplan, run_and_viz_pyperplan

from pyperplan import task

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
prep_browser()

<IPython.core.display.Javascript object>

# Explainability
As discussed in lecture, we know that _explaining_ unexpected behaviors from our robot can increase trust. In this module, we'll see how this comes into play. Recall our human-aware planning problem from lecture:

## Human-Aware Planning
Recall the Human-Aware Planning problem described in [1](https://arxiv.org/pdf/2105.01220.pdf).

**Input:**

$\mathcal{M}^R$, the robot's model of the environment and problem. Consists of the tuple $\langle\mathcal{D}^R, \mathcal{I}^R, \mathcal{G}^R\rangle$, where $\mathcal{D}^R$ is the domain,  $\mathcal{I}^R$ is the initial state, and $\mathcal{G}^R$ is the goal state.

$\mathcal{M}^G$, the human's model of the environment and problem. Consists of the tuple $\langle\mathcal{D}^H, \mathcal{I}^H, \mathcal{G}^H\rangle$, where $\mathcal{D}^H$ is the domain,  $\mathcal{I}^H$ is the initial state, and $\mathcal{G}^H$ is the goal state.

**Output:**

A _plan_; that is, a sequence of robot actions that achieve the goal state but also meets the human's expectations. We call the degree to which the robot plan $\pi$ matches the human expectations $\pi^e$ the plan _explicability_, and we often model it as the _distance_ $\delta$ between $\pi^e$ and $\pi$:

$$
E(\pi) = -1 * \delta(\pi^e, \pi)
$$

A plan $\pi$ is _perfectly explicable_ if $E(\pi) = 0$. We often use the difference in costs between the two plans as our distance function, $\delta$.


## Problem 1: Modeling Incomplete Knowledge
Because we're so familiar with PDDL, we will model both of these problems in PDDL! Check out the references from `ps-03` if you've forgotten how to use it. **TODO: describe this stuff in more detail.**

We'll start with a simplified version of the [Wumpus World](http://users.cecs.anu.edu.au/~patrik/pddlman/wumpus.html). Here, our robot is trying to navigate in a 3x3 grid to pick up a block. The robot knows that its shortest path is unencumbered, but the human has no idea--they think that there's piles of trash on the shortest path to the robot.

![Wumpus World](res/wumpus-world.png)

We've provided you with a fully modeled version of the robot world in `pddl/robot-domain.pddl` and `pddl/robot-problem.pddl`. Your task is to modify `pddl/human-domain.pddl` and `pddl/human-problem.pddl` so it reflects the fact that the _human_ thinks that there is trash in squares `(0, 2)` and `(1, 2)`. Notice that none of our actions are `durative-actions`; we assume a unit cost for every action.

Instead of using Optic to run our plans, we'll use a Python PDDL Planner called [`pyperplan`](https://github.com/aibasel/pyperplan). Pyperplan is cool because it is extensible and contains a very clean implementation of some of the commmon search algorithms; if you're interested in the way planners work, definitely check out their codebase! Like Optic, however, Pyperplan only supports _positive preconditions._  

Note, too, that we cannot specify negative initial conditions. This is important with our `clear` predicate. If we want to say that a square is not `clear` at the start, we simply omit it from the list of clear squares. See `robot-problem.pddl` for a concrete example.

In [47]:
domain_file = 'pddl/robot-domain.pddl'
problem_file = 'pddl/robot-problem.pddl'

run_pyperplan(domain_file, problem_file)

[<Op (robot-move robot1 sq2-2 sq2-1)>,
 <Op (robot-move robot1 sq2-1 sq1-1)>,
 <Op (robot-move robot1 sq1-1 sq0-1)>,
 <Op (robot-move robot1 sq0-1 sq0-2)>]

In [48]:
run_and_viz_pyperplan(domain_file, problem_file)

Plan found!

raw planner output:

(robot-move robot1 sq2-2 sq2-1)
  PRE: (at robot1 sq2-2)
  ADD: (at robot1 sq2-1)
  DEL: (at robot1 sq2-2)

(robot-move robot1 sq2-1 sq1-1)
  PRE: (at robot1 sq2-1)
  ADD: (at robot1 sq1-1)
  DEL: (at robot1 sq2-1)

(robot-move robot1 sq1-1 sq0-1)
  PRE: (at robot1 sq1-1)
  ADD: (at robot1 sq0-1)
  DEL: (at robot1 sq1-1)

(robot-move robot1 sq0-1 sq0-2)
  PRE: (at robot1 sq0-1)
  ADD: (at robot1 sq0-2)
  DEL: (at robot1 sq0-1)



How might we modify this domain to represent the human's belief that there is trash at position $(1, 2)$?

In [49]:
domain_file = 'pddl/human-domain.pddl'
problem_file = 'pddl/human-problem.pddl'

run_and_viz_pyperplan(domain_file, problem_file)

Plan found!

raw planner output:

(robot-move robot1 sq2-2 sq2-1)
  PRE: (at robot1 sq2-2)
  ADD: (at robot1 sq2-1)
  DEL: (at robot1 sq2-2)

(robot-move robot1 sq2-1 sq2-0)
  PRE: (at robot1 sq2-1)
  ADD: (at robot1 sq2-0)
  DEL: (at robot1 sq2-1)

(robot-move robot1 sq2-0 sq1-0)
  PRE: (at robot1 sq2-0)
  ADD: (at robot1 sq1-0)
  DEL: (at robot1 sq2-0)

(robot-move robot1 sq1-0 sq0-0)
  PRE: (at robot1 sq1-0)
  ADD: (at robot1 sq0-0)
  DEL: (at robot1 sq1-0)

(robot-move robot1 sq0-0 sq0-1)
  PRE: (at robot1 sq0-0)
  ADD: (at robot1 sq0-1)
  DEL: (at robot1 sq0-0)

(robot-move robot1 sq0-1 sq0-2)
  PRE: (at robot1 sq0-1)
  ADD: (at robot1 sq0-2)
  DEL: (at robot1 sq0-1)



## Problem 2: Plan Explainer

If you were the human in this human-robot team, you'd probably be concerned if your robot started moving towards where you thought was a gigantic pile of trash! This corresponds to a _decrease in trust_. As discussed in lecture, there are several ways to deal with these trust decreases:

**TODO: enumerate the ways trust can decrease**

Today, we'll be focusing on _explaining_; that is, telling our robot it's probably a good idea to explain its actions when it takes unexpected actions. We'll accomplish this by modifying the PDDL plan with an `explain` action that has some cost.

Let's try writing a function that will perform the robot's optimal plan, but explain to the human why their plans differ. We've implemented a few helper functions below to get you started, as well as outlined some pseudocode in the form of comments in the function `make_explainable_plan.` 

At a high-level, your function should take in as input an optimal robot plan and the expected human plan. It should then compare, action-by-action, the optimal action and the expected action, terminating when the optimal plan has reached the goal state. 

If the actions don't match up, you should check to see if the next optimal action is ever represented in the explainable plan. Then, you should skip to that action in the explainable plan and explain. 

**Todo: my implementation is certainly not robust enough and will need help debugging (i.e., what is the desired behavior if the human and robot plans converge? I think my code breaks in this case...)**

**Todo: Describe the desired output, as well as the helper functions, in greater detail**

In [50]:
def get_action_agent(operator):
    """
    Returns a string representing the agent completing some action
    Example: get_operation_agent(<Op (robot-move robot1 sq2-0 sq1-0)>)
            should return "robot-1".

    @param operator:    A pyperplan Operator object

    @return:            A string of the agent completing the operation 
    """
    return re.search("(?<=\s)(.*?)(?=\s)", operator.name).group()

def explain(agent):
    """
    Returns an Operator (explain `AGENT`)
    """
    return task.Operator('(explain ' + str(agent) + ')', frozenset(), frozenset(), frozenset())

def calculate_plan_cost(plan):
    """Calculates the cost of a plan. In our case (because each action has a unit cost), 
    the plan cost is simply the length of the plan. """
    return len(plan)

def compare_plans(optimal_plan, expected_plan):
    # Initialize explained plan to the empty list
    explained_plan = []

    # Iterate through index in optimal plan
    for i in range(len(optimal_plan)):
        # Get optimal and expected action at step i
        optimal_action = optimal_plan[i]
        expected_action = expected_plan[i]

        # Get the agent performing the optimal action
        agent = get_action_agent(optimal_action)

        # If the optimal action is what you'd expect, 
        # append the optimal action to the explained plan
        if optimal_action == expected_action:
            explained_plan.append(optimal_action)
        # Otherwise, try to find the index "next matching" plan action 
        else:
            try: 
                # See if the optimal action's add effects appear in the expected plan
                # If they do, remove all of the actions in the expected plan
                matching_index = [x.add_effects for x in expected_plan].index(optimal_action.add_effects)
                expected_plan = expected_plan[:i] + expected_plan[matching_index:]
                explained_plan.append(optimal_action)
            except:
                explained_plan.append(optimal_action)
                explained_plan.append(explain(agent))

    # Return the explained plan
    return explained_plan


In [51]:
human_domain_file = 'pddl/human-domain.pddl'
human_problem_file = 'pddl/human-problem.pddl'

robot_domain_file = 'pddl/robot-domain.pddl'
robot_problem_file = 'pddl/robot-problem.pddl'

expected_plan = run_pyperplan(human_domain_file, human_problem_file)
optimal_plan = run_pyperplan(robot_domain_file, robot_problem_file)
compare_plans(optimal_plan, expected_plan)

[<Op (robot-move robot1 sq2-2 sq2-1)>,
 <Op (robot-move robot1 sq2-1 sq1-1)>,
 <Op (explain robot1)>,
 <Op (robot-move robot1 sq1-1 sq0-1)>,
 <Op (robot-move robot1 sq0-1 sq0-2)>]

# The Meta-MDP

As discussed in class, one way of selecting robot behaviors can be modeled as a "Meta-MDP".